In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import tensorflow as tf

2024-03-10 13:29:15.137977: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('./dataset/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [6]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [7]:
from keras.callbacks import ModelCheckpoint
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [8]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [9]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
30/30 [==============================] - 7s 142ms/step - loss: 0.4127 - accuracy: 0.8474 - val_loss: 0.2717 - val_accuracy: 0.8608
Epoch 2/10
30/30 [==============================] - 4s 126ms/step - loss: 0.1729 - accuracy: 0.9488 - val_loss: 0.1059 - val_accuracy: 0.9726
Epoch 3/10
30/30 [==============================] - 4s 129ms/step - loss: 0.0722 - accuracy: 0.9794 - val_loss: 0.0740 - val_accuracy: 0.9778
Epoch 4/10
30/30 [==============================] - 4s 119ms/step - loss: 0.0463 - accuracy: 0.9855 - val_loss: 0.0852 - val_accuracy: 0.9800


In [10]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [11]:
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 0s 15ms/step - loss: 0.1093 - accuracy: 0.9725


In [12]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.109
  Accuracy: 0.972


In [13]:
tf.keras.models.save_model(model, 'my_model')

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [14]:
spam = "Free entry in 2 a weekly competition to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
pam = "Hi, how are you?"


sequences = tok.texts_to_sequences([spam])
predict_sequence = sequence.pad_sequences(sequences,maxlen=max_len)

# mpdel prediction
prediction = model.predict(predict_sequence)

# Output prediction results
# Usually the model output is close to 1 indicating spam, and close to 0 indicates non-spam. The threshold may need to be set according to the actual situation.
spam = prediction[0][0] > 0.5

if spam:
    print("The message is spam.")
else:
    print("The message is not spam.")

1/1 [==============================] - 0s 461ms/step
The message is spam.


In [15]:
import json
from keras.preprocessing.text import tokenizer_from_json
# Save Tokenizer to JSON format
tokenizer_json = tok.to_json() 
with open('tokenizer.json', 'w', encoding='utf-8') as f: 
    f.write(json.dumps(tokenizer_json, ensure_ascii=False)) 
    
# Load Tokenizer in JSON format
with open('tokenizer.json') as f: 
    tokenizer_data = json.load(f) 
    
loaded_tokenizer = tokenizer_from_json(tokenizer_data)

In [16]:
del model

# Load a model in SavedModel format
model = tf.keras.models.load_model('my_model')

In [17]:

spam = "Free entry in 2 a weekly competition to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
pam = "Hi, how are you?"

test = "Hello my name is Jucy How is your day going? Free entry in 2 a weekly competition to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

sequences = tok.texts_to_sequences([test])
predict_sequence = sequence.pad_sequences(sequences,maxlen=max_len)

prediction = model.predict(predict_sequence)

# Usually, the model output is close to 1, indicating spam, and close to 0, indicates non-spam. The threshold may need to be set according to the actual situation.
spam = prediction[0][0] > 0.5

if spam:
    print("The message is spam.")
else:
    print("The message is not spam.")

1/1 [==============================] - 1s 503ms/step
The message is spam.
